# Vector Search Evaluation with Qdrant
This notebook evaluates the performance of vector-based semantic search using Qdrant vector database and FastEmbed.

In [1]:
# Install required packages for Qdrant and FastEmbed
%pip install -q "qdrant-client[fastembed]>=1.14.2"

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import required libraries
import json
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import uuid
from qdrant_client import QdrantClient, models

### Setup Qdrant Connection
Initialize connection to Qdrant vector database

In [3]:
# Initialize Qdrant client (make sure Qdrant is running on localhost:6333 by cmd - docker run -p 6333:6333 qdrant/qdrant)

client = QdrantClient("http://localhost:6333")
print("Connected to Qdrant")
print("Existing collections:", client.get_collections())

Connected to Qdrant
Existing collections: collections=[]


### Load Data
Loading documents and ground truth data for evaluation

In [4]:
# Load documents from processed JSON file
with open('../data/processed/documents-with-ids.json', 'r') as f:
    documents = json.load(f)

# Load ground truth dataset for evaluation
df_ground_truth = pd.read_csv('../data/processed/ground-truth-retrieval.csv')
ground_truth = df_ground_truth.to_dict(orient='records')

print(f"Loaded {len(documents)} documents and {len(ground_truth)} ground truth questions")

Loaded 149 documents and 735 ground truth questions


### Configure Vector Search Model
Setup FastEmbed model configuration for embeddings

In [5]:
# Configure embedding model - using FastEmbed with jinaai model
model_name = "jinaai/jina-embeddings-v2-small-en"
embedding_dimensionality = 512

print(f"Using embedding model: {model_name}")
print(f"Embedding dimensionality: {embedding_dimensionality}")

Using embedding model: jinaai/jina-embeddings-v2-small-en
Embedding dimensionality: 512


### Create Qdrant Collection
Setup vector collection for storing document embeddings

In [6]:
# Define collection name with model identifier
collection_name = f"vector-search-{model_name.replace('/', '-')}"

# Delete collection if it already exists (for clean evaluation)
try:
    client.delete_collection(collection_name=collection_name)
    print(f"Deleted existing collection: {collection_name}")
except:
    print(f"Collection {collection_name} does not exist, creating new one")

Deleted existing collection: vector-search-jinaai-jina-embeddings-v2-small-en


In [7]:
# Create new collection with vector configuration
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=embedding_dimensionality,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

print(f"Created collection: {collection_name}")

Created collection: vector-search-jinaai-jina-embeddings-v2-small-en


### Upload Documents to Qdrant
Create embeddings and store documents in the vector database

In [8]:
print("Uploading documents to Qdrant with embeddings...")

# Prepare points for upload with FastEmbed embeddings
points = []
for idx, doc in enumerate(tqdm(documents, desc="Preparing points")):
    point = models.PointStruct(
        id=idx,  # Use index as point ID
        vector=models.Document(
            text=doc['content'], 
            model=model_name  # FastEmbed will generate embeddings automatically
        ),
        payload={
            "content": doc['content'],
            "id": doc['id'],
            "doc_id": doc.get('doc_id', ''),
            "location": doc.get('location', '')
        }
    )
    points.append(point)

# Upload points to Qdrant (FastEmbed will generate embeddings during upload)
client.upsert(
    collection_name=collection_name,
    points=points
)

print(f"Uploaded {len(points)} documents to Qdrant collection")

Uploading documents to Qdrant with embeddings...


Preparing points:   0%|          | 0/149 [00:00<?, ?it/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

c:\Users\Adi\Miniconda\envs\brahman_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Adi\AppData\Local\Temp\fastembed_cache\models--xenova--jina-embeddings-v2-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

onnx/model.onnx:   0%|          | 0.00/130M [00:00<?, ?B/s]

Uploaded 149 documents to Qdrant collection


### Define Search Function
Create function to perform vector search using Qdrant

In [9]:
def qdrant_vector_search(query, limit=5):
    """Perform vector search using Qdrant with FastEmbed embeddings"""
    
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_name  # FastEmbed generates query embedding automatically
        ),
        limit=limit,
        with_payload=True  # Include document metadata in results
    )
    
    return results

print("Vector search function defined")

Vector search function defined


### Test Search Function
Verify that the search is working correctly

In [17]:
# Test search with a sample query
test_query = "what is mysore famous for?"
test_results = qdrant_vector_search(test_query, limit=5)

print(f"Test query: {test_query}")
print(f"Found {len(test_results.points)} results:")
for i, result in enumerate(test_results.points, 1):
    print(f"{i}. Score: {result.score:.4f} | Content: {result.payload['content'][:100]}...")

Test query: what is mysore famous for?
Found 5 results:
1. Score: 0.9085 | Content: Mysore painting - Mysore is famous for the Mysore style of painting that is well known for its atten...
2. Score: 0.8687 | Content: the Kingdom of Mysore became an Indian state. The city of Mysore has so many surviving palaces that ...
3. Score: 0.8410 | Content: In the 3rd century BCE, the region we now know as Karnataka came under the Maurya emperors of Magadh...
4. Score: 0.8350 | Content: - the chief port city of Karnataka and second IT destination in Karnataka after Bangalore 6 Manipal ...
5. Score: 0.8233 | Content: ruled over much of what's now Karnataka and the larger Deccan Plateau from the mid-13th to the mid-1...


### Evaluation Process
Evaluate vector search performance using ground truth data

In [18]:
print("Evaluating vector search with Qdrant...")

# Initialize list to store relevance results
relevance_total = []

# Iterate through each ground truth question
for q in tqdm(ground_truth, desc="Evaluating retrieval"):
    doc_id = q['id']  # Ground truth document ID
    query = q['question']
    
    # Perform vector search using Qdrant
    search_results = qdrant_vector_search(query, limit=5)
    
    # Check if correct document is in results
    relevance = [point.payload['id'] == doc_id for point in search_results.points]
    relevance_total.append(relevance)

print("Evaluation completed!")

Evaluating vector search with Qdrant...


Evaluating retrieval:   0%|          | 0/735 [00:00<?, ?it/s]

Evaluation completed!


### Calculate Metrics
Compute Hit Rate and Mean Reciprocal Rank (MRR)

In [19]:
print("Calculating metrics...")

# Calculate Hit Rate
hit_count = sum(1 for line in relevance_total if True in line)
hit_rate = hit_count / len(relevance_total)

# Calculate Mean Reciprocal Rank (MRR)
total_score = 0.0
for line in relevance_total:
    for rank in range(len(line)):
        if line[rank] == True:
            total_score += 1 / (rank + 1)
            break

mrr = total_score / len(relevance_total)

# Create metrics dictionary
metrics = {
    'hit_rate': hit_rate,
    'mrr': mrr,
    'total_questions': len(relevance_total)
}

print(f"Metrics calculated successfully!")

Calculating metrics...
Metrics calculated successfully!


### Display Results
Show evaluation results before saving

In [20]:
# Display comprehensive results
print("\n" + "="*60)
print("VECTOR SEARCH EVALUATION RESULTS (with Qdrant)")
print("="*60)
print(f"Model: {model_name}")
print(f"Embedding Dimensions: {embedding_dimensionality}")
print(f"Collection: {collection_name}")
print(f"Distance Metric: COSINE")
print("-"*60)
print(f"Hit Rate: {metrics['hit_rate']:.4f} ({hit_count}/{len(relevance_total)})")
print(f"MRR (Mean Reciprocal Rank): {metrics['mrr']:.4f}")
print(f"Total Questions Evaluated: {metrics['total_questions']}")
print("-"*60)
print(f"Hit Rate %: {metrics['hit_rate']*100:.2f}%")
print("="*60)


VECTOR SEARCH EVALUATION RESULTS (with Qdrant)
Model: jinaai/jina-embeddings-v2-small-en
Embedding Dimensions: 512
Collection: vector-search-jinaai-jina-embeddings-v2-small-en
Distance Metric: COSINE
------------------------------------------------------------
Hit Rate: 0.5565 (409/735)
MRR (Mean Reciprocal Rank): 0.4144
Total Questions Evaluated: 735
------------------------------------------------------------
Hit Rate %: 55.65%


### Save Results
Save evaluation results to JSON file with model-specific naming

In [21]:
# Prepare results for saving with comprehensive information
results = {
    'method': 'vector_search_qdrant',
    'model_name': model_name,
    'model_config': {
        'embedding_dimensions': embedding_dimensionality,
        'distance_metric': 'COSINE',
        'vector_database': 'Qdrant',
        'embedding_provider': 'FastEmbed'
    },
    'collection_name': collection_name,
    'metrics': metrics,
    'relevance_results': relevance_total,
    'evaluation_details': {
        'total_documents': len(documents),
        'search_limit': 5,
        'evaluation_timestamp': pd.Timestamp.now().isoformat()
    }
}

In [25]:
# Create filename with model name for easy comparison
model_safe_name = model_name.replace('/', '_').replace('-', '_')
filename = f'../results/vector_search_qdrant_{model_safe_name}_results.json'

## Analysis Summary
Provide insights for model comparison

In [23]:
print("\n" + "="*60)
print("ANALYSIS SUMMARY")
print("="*60)
print(f"This evaluation used Qdrant vector database with FastEmbed")
print(f"for generating embeddings using {model_name}")
print()
print("Key Insights:")
print(f"• Hit Rate of {metrics['hit_rate']:.1%} means {hit_count} out of {len(relevance_total)} queries found relevant documents in top 5")
print(f"• MRR of {metrics['mrr']:.4f} indicates average rank position of first relevant result")
print(f"• Higher Hit Rate and MRR values indicate better search performance")
print()
print("For comparison with other models:")
print(f"• Check results files in 'results/' directory")
print(f"• Compare Hit Rate and MRR values across different models")
print(f"• Consider embedding dimensions vs performance trade-offs")
print("="*60)


ANALYSIS SUMMARY
This evaluation used Qdrant vector database with FastEmbed
for generating embeddings using jinaai/jina-embeddings-v2-small-en

Key Insights:
• Hit Rate of 55.6% means 409 out of 735 queries found relevant documents in top 5
• MRR of 0.4144 indicates average rank position of first relevant result
• Higher Hit Rate and MRR values indicate better search performance

For comparison with other models:
• Check results files in 'results/' directory
• Compare Hit Rate and MRR values across different models
• Consider embedding dimensions vs performance trade-offs


In [26]:
# Save results to JSON file
with open(filename, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved to: {filename}")
print(f"Collection '{collection_name}' remains in Qdrant for further analysis")


Results saved to: ../results/vector_search_qdrant_jinaai_jina_embeddings_v2_small_en_results.json
Collection 'vector-search-jinaai-jina-embeddings-v2-small-en' remains in Qdrant for further analysis
